In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate -U

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install sentence_transformers

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [ ]:
import pickle
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading LLama 2

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import transformers
import torch
# import bitsandbytes as bnb

name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    name,
    quantization_config=bnb_config,
    # torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,)


pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.float16,
    device_map="auto",
    # load_in_8bit=True,
    batch_size=4,
)

context = ""
user_prompt = '"Taste: sweet orange flavors Consistency: because it is neither too watery nor too hard" Can you extract clauses from this text?\n[/INST]?'

sequences = pipeline(
    context+user_prompt,
    do_sample=True,
    top_k=40,
    temperature=0.7,
    top_p=0.1,
    repetition_penalty=1.176,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Result: "Taste: sweet orange flavors Consistency: because it is neither too watery nor too hard" Can you extract clauses from this text?
[/INST]? Sure! Here are the clauses that I can identify in the given text:

1. Taste: sweet orange flavors
2. Consistency: because it is neither too watery nor too hard


In [ ]:
import pandas as pd

datapath = 'drive/MyDrive/Novacene/MAN_Merged_enriched_5.csv'
df = pd.read_csv(datapath)
df.head()

,SubmissionID,ClauseID,MemberID,Question,Statement Type,Statement,Feedback,Feedback Clauses,Feedback Clauses Cluster,Feedback Clauses Themes,Feedback Clauses Sentiment,Feedback Clauses Sentiment Scores
0,1,1,f212a989-9d71-4324-85c5-ed5b1024764c,MAN Q2_overarching,Overarching,"With savings, perks, and protection, your CAA ...",It implies that there is more to a CAA members...,It implies that there is more to a CAA members...,23,caa membership || the caa membership || a ca...,neutral,0.9991
1,2,1,680695fa-c698-4657-bc7e-ae3c011bbe84,MAN Q2_overarching,Overarching,"With savings, perks, and protection, your CAA ...",Sounds like there are additional benefits to m...,Sounds like there are additional benefits to m...,15,the membership benefits || membership benefit...,neutral,0.9979
2,3,1,5cfe8e6e-d83a-4b66-87c5-ae3c011d9964,MAN Q2_overarching,Overarching,"With savings, perks, and protection, your CAA ...",Feeling of safety,Feeling of safety,53,the protection aspect || some good protection...,neutral,0.9544
3,4,1,d09f925b-0928-40ca-b317-ae3c01234d75,MAN Q2_overarching,Overarching,"With savings, perks, and protection, your CAA ...",CAA is proud to provide their members with exc...,CAA is proud to provide their members with exc...,23,caa membership || the caa membership || a ca...,positive,0.9990
4,5,1,fd8bd845-b9dc-4646-952e-ae3c0134b5be,MAN Q2_overarching,Overarching,"With savings, perks, and protection, your CAA ...",Discounts while I travel,Discounts while I travel,1,discounts || special discounts || the discou...,neutral,0.9982


In [ ]:
df['Feedback Clauses'][252]+" ,"+df['Feedback Clauses'][253]+" ,"+df['Feedback Clauses'][254]+", "+df['Feedback Clauses'][255]

'I mainly am a member in the event I need car service or towing ,I also travel and on occasion obtain maps for the trips ,I don t really experience any savings that I am aware of and not sure what you mean by the statement that membership unlocks a world of good, That appears to be a stretch'

In [ ]:
feedback_dict = {}
max_clause_ids = df.groupby('SubmissionID')['ClauseID'].max().reset_index()
result_df = df.merge(max_clause_ids, on=['SubmissionID', 'ClauseID'], how='inner')
# feedback_dict['']

In [ ]:
result_df

,SubmissionID,ClauseID,MemberID,Question,Statement Type,Statement,Feedback,Feedback Clauses,Feedback Clauses Cluster,Feedback Clauses Themes,Feedback Clauses Sentiment,Feedback Clauses Sentiment Scores
0,1,1,f212a989-9d71-4324-85c5-ed5b1024764c,MAN Q2_overarching,Overarching,"With savings, perks, and protection, your CAA ...",It implies that there is more to a CAA members...,It implies that there is more to a CAA members...,23,caa membership || the caa membership || a ca...,neutral,0.9991
1,2,1,680695fa-c698-4657-bc7e-ae3c011bbe84,MAN Q2_overarching,Overarching,"With savings, perks, and protection, your CAA ...",Sounds like there are additional benefits to m...,Sounds like there are additional benefits to m...,15,the membership benefits || membership benefit...,neutral,0.9979
2,3,1,5cfe8e6e-d83a-4b66-87c5-ae3c011d9964,MAN Q2_overarching,Overarching,"With savings, perks, and protection, your CAA ...",Feeling of safety,Feeling of safety,53,the protection aspect || some good protection...,neutral,0.9544
3,4,1,d09f925b-0928-40ca-b317-ae3c01234d75,MAN Q2_overarching,Overarching,"With savings, perks, and protection, your CAA ...",CAA is proud to provide their members with exc...,CAA is proud to provide their members with exc...,23,caa membership || the caa membership || a ca...,positive,0.9990
4,5,1,fd8bd845-b9dc-4646-952e-ae3c0134b5be,MAN Q2_overarching,Overarching,"With savings, perks, and protection, your CAA ...",Discounts while I travel,Discounts while I travel,1,discounts || special discounts || the discou...,neutral,0.9982
...,...,...,...,...,...,...,...,...,...,...,...,...
623,624,1,8d3b74f2-9cf2-483f-9315-aedd00f00f5b,MAN Q16_loyalty2,Loyalty 2,"Use your CAA Membership for the savings, perks...",.,.,3,nil || sorry || lame || tells,neutral,0.9983
624,625,2,291720c0-67f2-47a9-a0c9-aedf0123f112,MAN Q16_loyalty2,Loyalty 2,"Use your CAA Membership for the savings, perks...",That CAA is friendly—the smiles,the smiles,16,the smiles statement || the smiles part || t...,positive,0.9996
625,626,5,6e88f911-93f2-4f97-8629-af16003311ef,MAN Q16_loyalty2,Loyalty 2,"Use your CAA Membership for the savings, perks...",If I don’t know what the savings etc are. Get...,CAA saves me at least 10 off list price on na...,23,caa membership || the caa membership || a ca...,neutral,0.9984
626,627,1,98c0736e-95ef-4f7e-8337-af1c00f1253d,MAN Q16_loyalty2,Loyalty 2,"Use your CAA Membership for the savings, perks...",I can use my CAA card for discounts on items o...,I can use my CAA card for discounts on items o...,23,caa membership || the caa membership || a ca...,neutral,0.9984


In [ ]:
grouped = result_df.groupby('Statement Type')
membership_df = grouped.get_group('Membership')
loyalty1_df = grouped.get_group('Loyalty 1')
loyalty2_df = grouped.get_group('Loyalty 2')

In [ ]:
import time

sys_prompt = """<s>[INST]<<SYS>>\n
I am trying to cluster customer survey responses based on whether they express the same themes. For each theme listed below, generate a comprehensive set of keyphrases that could describe the themes in the
text, as a JSON-formatted list.
These are the themes:\n
Canadian Automobile Association(CAA) Membership benefits\n
Money Savings\n
Roadside Assistance\n
Protection\n
\n<</SYS>>\n\n"""

context = ""
user_prompt = ' Can you extract the key phrases in the form of a JSON-formatted list without any explanations?\n[/INST]'

context = df['Feedback Clauses'][252]+" ,"+df['Feedback Clauses'][253]+" ,"+df['Feedback Clauses'][254]+", "+df['Feedback Clauses'][255]+"\n"

start_time = time.time() # Calculating start time

sequences = pipeline(
    sys_prompt+user_prompt,
    do_sample=True,
    top_k=40,
    temperature=0.7,
    top_p=0.1,
    repetition_penalty=1.176,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=200,
)
for seq in sequences:
    gen_response = seq['generated_text']
    latest_index = gen_response.rfind('[/INST]')
    result = gen_response[latest_index + len('[/INST]'):]
    print(f"Result: {result}")
    # clause_ext_dict['clauses_16bit']

end_time = time.time()
print("Execution Time:", end_time - start_time)

Result:   Of course! Here is a JSON-formatted list of potential keyphrases for each of the themes you provided:

{
"Canadian Automobile Association (CAA) Membership Benefits": ["membership", "benefits", "CAA", "auto club"],
"Money Savings": ["savings", "discounts", "deals", "financial savvy"],
"Roadside Assistance": ["roadside", "assistance", "towing", "breakdown"],
"Protection": ["protection", "security", "coverage", "insurance"]
}
Please note that these are just examples and may not be exhaustive or perfectly accurate representations of the themes you mentioned. The specific keyphrases used will depend on the context and content of your customer survey responses.
Execution Time: 15.136165380477905


### With Explanations

In [ ]:
import time

sys_prompt = """<s>[INST]<<SYS>>\n
I am trying to cluster customer survey responses based on whether they express the same themes. For each piece of input text, generate a
comprehensive set of keyphrases that aptly encapsulate the topics or the concept of the input text given to you while considering only those themes
(outlined below within the triple backticks) that might be relevant to the input text. Please consider only those themes that are relevant to the input
text and generate keyphrases accordingly and give the output in the form of a python list where the elements are separated by commas.
Generate a single set of keyphrases and do not make separate lists for each theme.\n
```
These are the themes:\n
Canadian Automobile Association(CAA) Membership benefits\n
Money Savings\n
Roadside Assistance\n
Protection\n
```
\n<</SYS>>\n\n"""

context = ""
user_prompt = ''' Please generate keyphrases only for those themes that are relevant to the content of the input text in the form of a python list where
each elements are separated by commas without any explanations.\n[/INST]'''


start_time = time.time() # Calculating start time
for feedback in result_df['Feedback'][20:30]:
  context = "input text: "+feedback+"\n[/INST]"
  sequences = pipeline(
      sys_prompt+context,
      do_sample=True,
      top_k=40,
      temperature=0.5,
      top_p=0.1,
      repetition_penalty=1.176,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=200,
  )
  for seq in sequences:
      gen_response = seq['generated_text']
      latest_index = gen_response.rfind('[/INST]')
      result = gen_response[latest_index + len('[/INST]'):]
      print("Text:", feedback)
      print(f"Result: {result}")
      # print(gen_response)
      print("------------------ENd------------------")

end_time = time.time()
print("Execution Time:", end_time - start_time)

Text: I can be assured that CAA has multiple ways to look after me and my needs 
Result:   Sure! Based on your input text, here are some keyphrases that capture the themes you provided:

* Money savings: "savings", "membership benefits", "cost-effective"
* Roadside assistance: "roadside help", "assistance", "towing"
* Protection: "protection", "security", "peace of mind"

So, the comprehensive set of keyphrases for your input text would be:

"savings", "membership benefits", "cost-effective", "roadside help", "assistance", "towing", "protection", "security", "peace of mind".
------------------ENd------------------
Text: Not much.  These isn’t any details provided…
Result:   Sure! Based on the input text "Not much. These isn't any details provided...", here are some keyphrases that could capture the main themes relevant to this text:

* Money savings (no details provided)
* Roadside assistance (not needed)
* Protection (from lack of details)

Note that these keyphrases are generated bas

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Text: Reasons to explore CAA more to see what is offered 
Result:   Sure! Based on the input text "Reasons to explore CAA more to see what is offered", here are some keyphrases that aptly encapsulate the themes relevant to this text:

* Money savings (e.g., discounts, deals, cost-effective solutions)
* Roadside assistance (e.g., emergency services, towing, tire changes)
* Protection (e.g., insurance, coverage, peace of mind)

So, the keyphrases generated for this input text are: money savings, roadside assistance, protection.
------------------ENd------------------
Execution Time: 151.94102597236633


In [ ]:
start_time = time.time() # Calculating start time
for feedback in membership_df['Feedback'][:20]:
  context = "input text: "+feedback+"\n[/INST]"
  sequences = pipeline(
      sys_prompt+context,
      do_sample=True,
      top_k=40,
      temperature=0.5,
      top_p=0.1,
      repetition_penalty=1.176,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=200,
  )
  for seq in sequences:
      gen_response = seq['generated_text']
      latest_index = gen_response.rfind('[/INST]')
      result = gen_response[latest_index + len('[/INST]'):]
      print("Text:", feedback)
      print(f"Result: {result}")
      # print(gen_response)
      print("------------------ENd------------------")

end_time = time.time()
print("Execution Time:", end_time - start_time)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Text: It's just another advertising slogan.  I don't really care about it one way or another.
Result:   Sure! Based on the input text "It's just another advertising slogan. I don't really care about it one way or another," here are some keyphrases that could capture the themes relevant to this text:

* Advertising slogans
* Indifference towards marketing messages
* Lack of interest in promotional materials
* Disconnection from commercial appeals

These keyphrases aim to summarize the main ideas conveyed in the input text while focusing on the themes outlined above.
------------------ENd------------------
Text: I can save money with the membership. 
Result:   Sure! Based on the input text "I can save money with the membership," here are some keyphrases that aptly encapsulate the themes relevant to this text:

* Money savings
* CAA membership benefits
* Cost savings
* Financial protection

Note that these keyphrases are generated based on the themes outlined in your request, which are Ca

In [ ]:
start_time = time.time() # Calculating start time
for feedback in loyalty1_df['Feedback'][:20]:
  context = "input text: "+feedback+"\n[/INST]"
  sequences = pipeline(
      sys_prompt+context,
      do_sample=True,
      top_k=40,
      temperature=0.5,
      top_p=0.1,
      repetition_penalty=1.176,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=200,
  )
  for seq in sequences:
      gen_response = seq['generated_text']
      latest_index = gen_response.rfind('[/INST]')
      result = gen_response[latest_index + len('[/INST]'):]
      print("Text:", feedback)
      print(f"Result: {result}")
      # print(gen_response)
      print("------------------ENd------------------")

end_time = time.time()
print("Execution Time:", end_time - start_time)

Text: It's nor properly formatted, grammar-wise.  At least not in my opinion.'Smiles' does not resonate with me.  I only care about savings, perks and vehicle breakdown protection etc.
Result:   Sure! Based on your input text, here is a comprehensive set of keyphrases that capture the main themes you mentioned:

Keyphrases:
Money savings, CAA membership benefits, roadside assistance, vehicle protection, breakdown protection, perks

Explanation:
The input text mentions that the "smile" aspect of the CAA membership doesn't resonate with them, so we can infer that they are more interested in the financial aspects of the membership such as money savings. Additionally, they mention "perks," which could refer to any additional benefits offered by the CAA. The other themes mentioned in the prompt (roadside assistance, vehicle protection, and breakdown protection) are also relevant to the input text since they relate to the idea of being protected on the road.
------------------ENd------------

In [ ]:
start_time = time.time() # Calculating start time
for feedback in loyalty2_df['Feedback'][:20]:
  context = "input text: "+feedback+"\n[/INST]"
  sequences = pipeline(
      sys_prompt+context,
      do_sample=True,
      top_k=40,
      temperature=0.5,
      top_p=0.1,
      repetition_penalty=1.176,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=200,
  )
  for seq in sequences:
      gen_response = seq['generated_text']
      latest_index = gen_response.rfind('[/INST]')
      result = gen_response[latest_index + len('[/INST]'):]
      print("Text:", feedback)
      print(f"Result: {result}")
      # print(gen_response)
      print("------------------ENd------------------")

end_time = time.time()
print("Execution Time:", end_time - start_time)

Text: The statement does not read well, I don’t think the emphasis on the club is completely relevant to what CAA represents.
Result:   Sure! Based on the input text you provided, here are some keyphrases that could capture the main themes related to the topic:

* Canadian Automobile Association (CAA) membership benefits
	+ Roadside assistance
	+ Protection
	+ Money savings
* Relevance of the club to CAA representation
	+ Not entirely relevant
	+ Club relevance
	+ Emphasis on club

Note that these keyphrases are generated based on the information provided in the input text, and may not necessarily cover all possible aspects of the topic.
------------------ENd------------------
Text: I might be able to use my CAA card to get a deal on a purchase
Result:   Sure! Based on the input text "I might be able to use my CAA card to get a deal on a purchase", here are some keyphrases that aptly encapsulate the topics or concepts related to Canadian Automobile Association (CAA) membership benefits

### W/O Explanations and with User prompt

In [ ]:
import time

sys_prompt = """<s>[INST]<<SYS>>\n
I am trying to cluster this text based on whether they discuss the same topics. For each topic, listed in the triple backticks below,
generate a comprehensive set of keyphrases that could describe the themes in the text. Please consider only thoses themes that are relevant
to the input text and generate keyphrases accordingly.

```
These are the themes:\n
Canadian Automobile Association(CAA) Membership benefits\n
Money Savings\n
Roadside Assistance\n
Protection\n
```

\n<</SYS>>\n\n"""

context = ""
user_prompt = ''' Please consider only for those themes that are relevant to the content of the input text and give the key-phrases in the
form of a python list where the keyphrases are the elements of the list and the elements are separated by commas without any explanations.\n[/INST]'''


start_time = time.time() # Calculating start time
for feedback in result_df['Feedback'][:2]:
  context = "input text: "+feedback+"\n[/INST]"
  sequences = pipeline(
      sys_prompt+context,
      do_sample=True,
      top_k=40,
      temperature=0.5,
      top_p=0.1,
      repetition_penalty=1.176,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=200,
  )
  for seq in sequences:
      gen_response = seq['generated_text']
      latest_index = gen_response.rfind('[/INST]')
      result = gen_response[latest_index + len('[/INST]'):]
      print("Text:", feedback)
      print(f"Result: {result}")
      # print(gen_response)

end_time = time.time()
print("Execution Time:", end_time - start_time)

Text: It implies that there is more to a CAA membership than just vehicle breakdown benefits, which is all I thought CAA was good for before I Actually joined
Result:   Sure! Based on the provided themes and the input text, here's a comprehensive set of keyphrases that could describe the themes in the text:
* Canadian Automobile Association (CAA) Membership benefits: "CAA Membership", "Benefits", "Discounts", "Savings"
* Money Savings: "Savings", "Money", "Discounts", "Financial security"
* Roadside Assistance: "Assistance", "Help", "Insurance", "Assurance"
* Protection: "Protection", "Feelings of safety", "Security", "Peace of mind"

Here's an example of how these keyphrases could be used to summarize the main ideas in the input text:
Input Text: Your CAA membership will unlock discounts and savings for you.
Summary Keyphrases: C
Text: Sounds like there are additional benefits to my membersho
Result:   Sure! Based on the provided themes and input text, here's a comprehensive set of ke

In [ ]:
result_df['Feedback'][18]

'Benifits such as towing, battery checks, trip tics, insurance, maps,discounts'

### Prompt template from Author's codebase

In [ ]:
Sys prompt = f"I am trying to cluster tweets based on whether " + \
            "they discuss the same topic. To do this, given a (stopword-removed) tweet, please " + \
            "provide a comprehensive set of keywords or keyphrases that could describe this tweet's " + \
            "topic. These keywords should be distinct from those that might describe tweets with different " + \
            "topics. Since the tweets already look like keywords, feel free to include keywords not listed " + \
            "in the tweet, and don't feel like you need to include very many of the original words from the " + \
            "tweet. Generate a comprehensive set of keyphrases as a JSON-formatted list."


sentences = [
                    "brain fluid buildup delay giffords rehab",
                    "trailer talk week movie rite mechanic week opportunity",
                    "gbagbo camp futile cut ivory coast economy",
                    "chicken cavatelli soup"
                ]
                keywords = [
                    ["gabrielle giffords", "giffords recovery"],
                    ["movies", "in theaters", "trailer talk"],
                    ["gbagbo", "ivory coast"],
                    ["cooking", "tasty recipes"]
                ]

Canadian Automobile Association(CAA) Membership benefits\n
Money Savings\n
Roadside Assistance\n
Protection\n

### Generating Key-phrases with selective Themes


In [ ]:
import time

sys_prompt = """<s>[INST]<<SYS>>\n
Your task is to extract keyphrases that aptly encapsulate the themes outlined below within the triple backticks. Please consider only those themes that are relevant to the input text and generate keyphrases
accordingly. Give the output in the form of a python list where the elements are separated by commas and consider only the information provided to you in the form of input text as context.\n
```
These are the themes:\n
Roadside Assistance\n
```
\n<</SYS>>\n\n"""

context = ""
user_prompt = ''' Please generate keyphrases only for those themes that are relevant to the content of the input text in the form of a python list where
each elements are separated by commas without any explanations.\n[/INST] '''


start_time = time.time() # Calculating start time
# for feedback in result_df['Feedback'][0:10]:
context = "input text: "+result_df['Feedback'][18]+"\n[/INST]"
sequences = pipeline(
    sys_prompt+context,
    do_sample=True,
    top_k=40,
    temperature=0.5,
    top_p=0.1,
    repetition_penalty=1.176,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=200,
)
for seq in sequences:
    gen_response = seq['generated_text']
    latest_index = gen_response.rfind('[/INST]')
    result = gen_response[latest_index + len('[/INST]'):]
    # print(f"Result: {result}")
    print(gen_response)
    print("------------------END------------------")

end_time = time.time()
print("Execution Time:", end_time - start_time)

<s>[INST]<<SYS>>

Your task is to extract keyphrases that aptly encapsulate the themes outlined below within the triple backticks. Please consider only those themes that are relevant to the input text and generate keyphrases 
accordingly. Give the output in the form of a python list where the elements are separated by commas and consider only the information provided to you in the form of input text as context.

```
These are the themes:

Roadside Assistance

``` 

<</SYS>>

input text: Benifits such as towing, battery checks, trip tics, insurance, maps,discounts
[/INST]  Sure! Based on the input text, here are some potential keyphrases that encapsulate the themes related to roadside assistance:
* Towing services
* Battery check services
* Trip tickets or travel discounts
* Insurance services for roadside emergencies
* Maps and navigation tools for finding help

I hope these keyphrases are helpful! Let me know if you have any further questions or need additional assistance.
-----------

In [ ]:
result_df['Feedback'][0]

'It implies that there is more to a CAA membership than just vehicle breakdown benefits, which is all I thought CAA was good for before I Actually joined'

### Author's prompt

In [ ]:
import time

sys_prompt = f"I am trying to cluster survey responses based on whether " + \
            "they discuss the same topic. To do this, given a survey response(as an input text), please " + \
            "provide a comprehensive set of keywords or keyphrases that could describe this input text's " + \
            "topic. Your task is to consider the themes(provided below in the triple backticks) that are relevant to the input text" + \
            "and generate keywords or key phrases that encapsulate the content of the the relevant themes along with the content of the input text." + \
            "Please generate keywords or keyphrases only for those themes that are relevant" + \
            "to the content or topic of the input text based on the examples provided below in the examples. These keywords or keyphrases should be" + \
            "be distinct from those that might describe the input text with different " + \
            "topics. Please do not give any explanations for the keywords and follow the examples given to you. Since the text already looks like keywords or" + \
            "keyphrases, feel free to include the ones that are not listed in the input text, but you are supposed to generate only a single output with a" + \
            "single list of keyphrases for input text that is" + \
            "being given to you alone, don't make separate lists for each theme." + \
            "\n" + \
              """
              ```
              These are the themes:\n
              Canadian Automobile Association(CAA) Membership benefits\n
              Money Savings\n
              Roadside Assistance\n
              Protection\n
              ```
              input text: Your CAA membership will unlock discounts and savings for you\n
              Keyphrases: “CAA Membership, CAA Benefits, savings, discounts”\n\n
              input text: it also offers savings at a variety of businesses\n
              Keyphrases: “Savings, money, discounts at stores, discounts at businesses”\n\n
              input text: when and if you do  they will be there to help you in any way that is possible and you are so glad that you do carry that kind of insurance assurance when you do need it most\n
              Keyphrases: “Roadside assistance, help, insurance, assurance”\n\n

              """


context = ""
user_prompt = ''' \n[/INST] '''


start_time = time.time() # Calculating start time
for feedback in result_df['Feedback'][10:20]:
  context = "input text: "+feedback+"\n[/INST]"
  sequences = pipeline(
      sys_prompt+context,
      do_sample=True,
      top_k=40,
      temperature=0.5,
      top_p=0.1,
      repetition_penalty=1.176,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=200,
  )
  for seq in sequences:
      gen_response = seq['generated_text']
      latest_index = gen_response.rfind('[/INST]')
      result = gen_response[latest_index + len('[/INST]'):]
      print("Text:", feedback)
      print(f"Result: {result}")
      # print(gen_response)
      print("------------------ENd------------------")

end_time = time.time()
print("Execution Time:", end_time - start_time)

### Few Shot Prompting

In [ ]:
import time

sys_prompt = """<s>[INST]<<SYS>>\n
Your task is to extract keyphrases that aptly encapsulate the topics or the concept of the input text given to you while considering only those themes
(outlined below within the triple backticks) that might be relevant to the input text. Give the output in the form of a python list where the elements are separated by
commas and consider only the information provided to you in the form of input text as context. Two different examples are being given to
you as demonstrations, please generate keyphrases as shown in these examples. You don't have to use the same keyphrases listed in the
examples. Feel free to list keyphrases that are not mentioned in the examples, but are still relevant to the topic of the
input text and the themes. You are supposed to generate only a single output with one single list of keyphrases for input text that is
being given to you alone, don't make separate lists for each theme.\n
```
These are the themes:\n
Canadian Automobile Association(CAA) Membership benefits\n
Money Savings\n
Roadside Assistance\n
Protection\n
```
\n<</SYS>>\n\n"""

context = ""
user_prompt = ''' Please generate keyphrases only for those themes that are relevant to the content of the input text in the form of a python list where
each elements are separated by commas without any explanations.\n[/INST]'''


start_time = time.time() # Calculating start time
for feedback in result_df['Feedback'][:20]:
  context = "input text: "+feedback+"\n[/INST]"
  sequences = pipeline(
      sys_prompt+context,
      do_sample=True,
      top_k=40,
      temperature=0.5,
      top_p=0.1,
      repetition_penalty=1.176,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=300,
  )
  for seq in sequences:
      gen_response = seq['generated_text']
      latest_index = gen_response.rfind('[/INST]')
      result = gen_response[latest_index + len('[/INST]'):]
      print("Text:", feedback)
      print(f"Result: {result}")
      # print(gen_response)
      print("------------------End------------------")

end_time = time.time()
print("Execution Time:", end_time - start_time)

Text: It implies that there is more to a CAA membership than just vehicle breakdown benefits, which is all I thought CAA was good for before I Actually joined
Result:   Sure! Based on the input text you provided, here are some keyphrases that accurately capture the main topics discussed:
* CAA Membership
* Benefits
* Savings
* Discounts

I extracted these phrases because they are directly related to the themes outlined earlier (CAA Membership benefits, savings, roadside assistance, protection), and they appear frequently throughout the input text.
------------------ENd------------------
Text: Sounds like there are additional benefits to my membersho
Result:   Sure! Based on the input text you provided, here are some keyphrases that accurately capture the main topics discussed:
* "CAA Membership"
* "Benefits"
* "Savings"
* "Discounts"
* "Roadside Assistance"
* "Protection"

These keyphrases cover all the major themes outlined in your request, including Canadian Automobile Association (C

In [ ]:
import time

sys_prompt = """<s>[INST]<<SYS>>\n
Your task is to extract keyphrases that aptly encapsulate the topics or the concept of the input text given to you while considering only those themes
(outlined below within the triple backticks) that might be relevant to the input text. Please consider only those themes that are relevant
to the input text and generate keyphrases accordingly. Give the output in the form of a python list where the elements are separated by
commas and consider only the information provided to you in the form of input text as context. Two different examples are being given to
you as demonstrations, please generate keyphrases as shown in these examples. You don't have to use the same keyphrases listed in the
examples. Feel free to list keyphrases that are not mentioned in the examples, but are still relevant to the topic of the
input text and the themes. You are supposed to generate only a single output with one single list of keyphrases for input text that is
being given to you alone, don't make separate lists for each theme.\n

These are the themes:\n
Canadian Automobile Association(CAA) Membership benefits\n
Money Savings\n
Roadside Assistance\n
Protection\n
Here are the examples: \n
input text: Your CAA membership will unlock discounts and savings for you\n
Keyphrases: “CAA Membership, CAA Benefits, savings, discounts”\n
input text: it also offers savings at a variety of businesses\n
Keyphrases: “Savings, money, discounts at stores, discounts at businesses”\n


\n<</SYS>>\n\n"""

context = ""
user_prompt = ''' Please generate keyphrases only for those themes that are relevant to the content of the input text in the form of a python list where
each elements are separated by commas without any explanations.\n[/INST]'''


start_time = time.time() # Calculating start time
for feedback in result_df['Feedback'][:20]:
  context = "input text: "+feedback+"\n[/INST]"
  sequences = pipeline(
      sys_prompt+context,
      do_sample=True,
      top_k=40,
      temperature=0.5,
      top_p=0.1,
      repetition_penalty=1.176,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=300,
  )
  for seq in sequences:
      gen_response = seq['generated_text']
      latest_index = gen_response.rfind('[/INST]')
      result = gen_response[latest_index + len('[/INST]'):]
      print("Text:", feedback)
      print(f"Result: {result}")
      # print(gen_response)
      print("------------------End------------------")

end_time = time.time()
print("Execution Time:", end_time - start_time)

Text: It implies that there is more to a CAA membership than just vehicle breakdown benefits, which is all I thought CAA was good for before I Actually joined
Result:   Sure! Based on the input text you provided, here are some keyphrases that accurately capture the main ideas and concepts:
* CAA Membership
* Benefits
* Savings
* Discounts
* Roadside Assistance

I extracted these phrases from your input text because they are directly related to the themes you outlined earlier (Canadian Automobile Association membership benefits, money savings, roadside assistance, and protection). These phrases help summarize the main points of the input text without going into too much detail.
------------------ENd------------------
Text: Sounds like there are additional benefits to my membersho
Result:   Sure! Based on the input text you provided, here are some keyphrases that accurately capture the main topics and concepts discussed:
* "CAA Membership"
* "Benefits"
* "Savings"
* "Discounts"
* "Roadsi

Automating Generating keyphrases and appending it to the excel file.

In [ ]:
import time
responses = []
sys_prompt = """<s>[INST]<<SYS>>\n
I am trying to cluster customer survey responses based on whether they express the same themes. Your task is to extract keyphrases that aptly encapsulate the topics or the concept of the input text given to you while considering only those themes
(outlined below within the triple backticks) that might be relevant to the input text. You are supposed to generate a single output for the response presented to you and produce a single list of keyphrases,
don't make separate lists for each theme. You don't have to use the same keyphrases listed in the examples. You are not supposed to generate any new information or responses.
Feel free to list keyphrases that are not mentioned in the examples, but are still relevant to the topic of the input text and the themes. Only generate keyphrases for the latest
response and end your answer. Give the output in the form of a python list where the elements are separated by commas.\n

```
These are the themes:\n
Canadian Automobile Association(CAA) Membership benefits\n
Money Savings\n
Roadside Assistance\n
Protection\n
```
\n<</SYS>>\n\n"""

context = ""
user_prompt = ''' Please generate keyphrases only for those themes that are relevant to the content of the input text in the form of a python list where
each elements are separated by commas without any explanations.\n[/INST]'''


start_time = time.time() # Calculating start time
for feedback in result_df['Feedback'][0:5]:
  context = "input text: "+feedback+"\n[/INST]"
  sequences = pipeline(
      sys_prompt+context,
      do_sample=True,
      top_k=40,
      temperature=0.5,
      top_p=0.1,
      repetition_penalty=1.176,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=200,
  )
  for seq in sequences:
      gen_response = seq['generated_text']
      latest_index = gen_response.rfind('[/INST]')
      result = gen_response[latest_index + len('[/INST]'):]
      print("Text:", feedback)
      print(f"Result: {result}")
      responses.append(result)
      # print(gen_response)
      print("------------------ENd------------------")

end_time = time.time()
print("Execution Time:", end_time - start_time)

Text: It implies that there is more to a CAA membership than just vehicle breakdown benefits, which is all I thought CAA was good for before I Actually joined
Result:   Sure! Based on the input text, here are some keyphrases that aptly encapsulate the themes:

* "CAA membership"
* "vehicle breakdown benefits"
* "money savings"
* "roadside assistance"
* "protection"

These keyphrases capture the main ideas expressed in the input text, including the broader range of services offered by a CAA membership beyond just roadside assistance.
------------------ENd------------------
Text: Sounds like there are additional benefits to my membersho
Result:   Sure! Based on the input text "Sounds like there are additional benefits to my membership," I will extract the following keyphrases that are relevant to the themes outlined above:

* Money savings
* Roadside assistance
* Protection

These keyphrases capture the main ideas expressed in the input text related to the CAA membership benefits.
------

In [ ]:
responses = result

In [ ]:
import time


context = ""
user_prompt = ''' Please convert the bullet points in the response into list of keywords separated by comma.\n[/INST]'''
keyphrases = []

start_time = time.time() # Calculating start time
for result in responses:
  context = result+"\n[/INST]"
  sequences = pipeline(
      context+user_prompt,
      do_sample=True,
      top_k=40,
      temperature=0.5,
      top_p=0.1,
      repetition_penalty=1.176,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=200,
  )
  for seq in sequences:
      gen_response = seq['generated_text']
      latest_index = gen_response.rfind('[/INST]')
      result_list = gen_response[latest_index + len('[/INST]'):]
      keyphrases.append(result_list)
      print("Text:", feedback)
      print(f"Result: {result_list}")
      # print(gen_response)
      print("------------------ENd------------------")

end_time = time.time()
print("Execution Time:", end_time - start_time)

### Using Centroid for Generating Keyphrases

In [ ]:
datapath = 'drive/MyDrive/Novacene/MAN_Merged_enriched_5.csv'
df = pd.read_csv(datapath)

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import SentenceTransformer
import numpy as np

import warnings
warnings.filterwarnings('ignore')

class Clustering:

    def __init__(self, model='fast', cluster_threshold=0.70):

        if model == 'base':
            self.encode_model = 'stsb-roberta-base'
        else:
            self.encode_model =  'all-MiniLM-L6-v2'
        self.cluster_threshold = float(cluster_threshold)
        self.clustering_model = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=self.cluster_threshold)

    def order_dict_by_index(self, my_dict, index_list):
        # create a new dictionary to store the sorted key-value pairs
        sorted_dict = {}
        # loop through the index list
        for key_index in index_list:
            # check if the key is in the dictionary
            if key_index in my_dict:
                # add the key-value pair to the new dictionary
                sorted_dict[key_index] = my_dict[key_index]
        # return the sorted dictionary
        return sorted_dict

    def get_cosim(self, centroid,sent_vector):

      sim_scores = centroid.dot(sent_vector.T)

      return sim_scores

    def centroid(self, vectors):
      """Compute the centroid of a list of vectors"""

      dim = len(vectors[0])
      centroid = [0.0] * dim
      for vector in vectors:
          for i in range(dim):
              centroid[i] += vector[i]
      centroid = [coord / len(vectors) for coord in centroid]
      return centroid

    def get_unique(self, data_list):
        unique_data_list = []
        duplicate_dict = {}
        for i in range(len(data_list)):
            element = data_list[i]
            if element in duplicate_dict:
                dup_list = duplicate_dict[element]
                dup_list.append(i)
                duplicate_dict[element] = dup_list
            else:
                unique_data_list.append(element)
                duplicate_dict[element] = [i]
        return unique_data_list, duplicate_dict

    def find_ind(self, lst, val):
        """
        Returns a list of indexes where the specified value is found in the list.
        """
        indexes = []
        for i in range(len(lst)):
            if lst[i] == val:
                indexes.append(i)

        return indexes

    def copy_row_to_array(self,text_data_list, sent_vectors, duplicate_dict):
        new_array = np.zeros((len(text_data_list),np.shape(sent_vectors)[1]))
        for i, key, values in zip(range(len(sent_vectors)), duplicate_dict.keys(), duplicate_dict.values()):
            for index in values:
                new_array[index,:] = sent_vectors[i,:]

        return new_array

    def get_cluster_meta_data(self, predicted, sent_vectors, data_list, handle_duplicates, duplicate_dict):
        clusters = {}
        clusters_unique = list(np.unique(predicted))
        central_docs = {}
        central_docs_vector = {}
        dist_to_central = {}
        cluster_meta = {}
        cluster_meta_reconstruct = {}

        if handle_duplicates == True:
            sent_vectors = self.copy_row_to_array(data_list, sent_vectors, duplicate_dict)

        for cluster in clusters_unique:
            clusters[cluster] = self.find_ind(predicted, cluster)

        for key, value in zip(clusters.keys(), clusters.values()):
            vectors = sent_vectors[value]
            cent = self.centroid(vectors)

            central = value[np.argmax(self.get_cosim(np.array(cent), np.array(vectors)))]
            central_docs_vector[key] = sent_vectors[central]


            cluster_meta[key] = {'central_member': central,
                                'central_vector': sent_vectors[central],
                                'distance_to_central': self.get_cosim(np.array(central_docs_vector[key]), np.array(vectors)),
                                'member_indices': value}

        for key, value in zip(cluster_meta.keys(), cluster_meta.values()):
            for j,k in zip(cluster_meta[key]['distance_to_central'], cluster_meta[key]['member_indices']):
                cluster_meta_reconstruct[k] = {'cluster':key, 'distance_to_central': j}

        return self.order_dict_by_index(cluster_meta_reconstruct, list(range(len(sent_vectors))))
        # return cluster_meta

    def get_clusters(self, text_data_list):

        embedder = SentenceTransformer(self.encode_model)
        corpus_embeddings = embedder.encode(text_data_list)

        corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

        self.clustering_model.fit(corpus_embeddings)

        cluster_assignment = list(self.clustering_model.labels_)

        return cluster_assignment, corpus_embeddings

def convert_to_string(data_list):
    for i in range(len(data_list)):
        data_list[i] = str(data_list[i])
    return data_list

In [ ]:
data_list = convert_to_string(df['Feedback Clauses'])
clusterer = Clustering(model ='fast', cluster_threshold= float('0.7'))
predicted, sent_vectors = clusterer.get_clusters(data_list)
unique_data_list, duplicate_dict = clusterer.get_unique(data_list)
meta_data = clusterer.get_cluster_meta_data(predicted, sent_vectors, data_list, True, duplicate_dict)

In [ ]:
meta_data

In [ ]:
from collections import defaultdict
closest_points = defaultdict(list)
for idx, data in meta_data.items():
    cluster = data['cluster']
    distance = data['distance_to_central']
    # Append the index and distance to a list for each cluster
    closest_points[cluster].append((idx, distance))

In [ ]:
centroid_samples = {}
for cluster, points in closest_points.items():
    # Sort the points by distance
    points.sort(key=lambda x: x[1])
    # Select the point closest to the centroid
    index, _ = points[0]
    centroid_samples[cluster] = index

In [ ]:
centroid_samples

In [ ]:
for cluster in centroid_samples:
  print(df['Feedback Clauses'][centroid_samples[cluster]])

In [ ]:
len(centroid_samples)

71

In [ ]:
df['Feedback Clauses'][centroid_samples[0]]

'It may be true'

### Sister Merci Dataset

In [ ]:
import pandas as pd

datapath = 'drive/MyDrive/Novacene/flower - for analysis v4_enriched_9.csv'
# datapath = 'drive/MyDrive/Novacene/enriched_flowers_keywords_few_shot.csv'
# datapath = 'drive/MyDrive/Novacene/sister_merci_dataset.csv'
flowers_df = pd.read_csv(datapath)
flowers_df.head()

,SubmissionID,ClauseID,Title with Quotes,Content with Quotes,Subreddit,Sentiment,URL,Date,Brand 1,Subreddit Cleansed,Content with Quotes Cleansed,Content with Quotes Cleansed Translated,Content with Quotes Cleansed Translated Clauses,Content with Quotes Cleansed Translated Clauses Cluster,Content with Quotes Cleansed Translated Clauses Themes
0,1,1,"""King Tut 17.3% Petits Plaisirs""","""""",sqdc,POSITIVE,https://www.reddit.com/r/sqdc/comments/p7twdq,2021-08-20,NaN,sqdc,[Empty],[Empty],Empty,11,but nothing || so nothing || absolutely noth...
1,2,1,"""48 North Green Crush -&gt; Bag of Schwag WHY ...","""I saw other people post nice pictures of this...",sqdc,NEGATIVE,https://www.reddit.com/r/sqdc/comments/p7l0w3,2021-08-20,NaN,sqdc,I saw other people post nice pictures of this ...,I saw other people post nice pictures of this ...,I saw other people post nice pictures of this ...,525,a 3 5 g purchase || 5gs pricing || 3 5gs ||...
2,2,2,"""48 North Green Crush -&gt; Bag of Schwag WHY ...","""I saw other people post nice pictures of this...",sqdc,NEGATIVE,https://www.reddit.com/r/sqdc/comments/p7l0w3,2021-08-20,NaN,sqdc,I saw other people post nice pictures of this ...,I saw other people post nice pictures of this ...,It was predominantly leaves and stems with no ...,416,a few stems || small stems || many stems ||...
3,2,3,"""48 North Green Crush -&gt; Bag of Schwag WHY ...","""I saw other people post nice pictures of this...",sqdc,NEGATIVE,https://www.reddit.com/r/sqdc/comments/p7l0w3,2021-08-20,NaN,sqdc,I saw other people post nice pictures of this ...,I saw other people post nice pictures of this ...,I even tested it,670,further testing || the testing || the tests ...
4,2,4,"""48 North Green Crush -&gt; Bag of Schwag WHY ...","""I saw other people post nice pictures of this...",sqdc,NEGATIVE,https://www.reddit.com/r/sqdc/comments/p7l0w3,2021-08-20,NaN,sqdc,I saw other people post nice pictures of this ...,I saw other people post nice pictures of this ...,it had less than 6 advertised as 19 29 THC,255,Cannabis Thc || a 15 thc weed || 35 thc we...


In [ ]:
flowers_df['Content with Quotes Cleansed Translated Clauses'][1]

'I saw other people post nice pictures of this product  and bought a 3 5g from Pointe Claire SQDC of 48 North Green Crush  August 2021'

In [ ]:
import time

sys_prompt = """<s>[INST]<<SYS>>\n
Your task is to extract keyphrases that aptly encapsulate the topics or the concept of the input text given to you. You are supposed to generate a single output for the input text presented to you and produce a single list
of keyphrases. You don't have to use the same keyphrases listed in the examples and don't give any explanations. You are not supposed to generate any new information or input texts.
Feel free to list keyphrases that are not mentioned in the examples, but are still relevant to the topic of the input text.

Here are the examples given to you as demonstrations: \n
input text: I really enjoy It, but at the price, it will most likely be a one off for me.\n
Keyphrases: “Price, affordability”\n
input text: The aroma smacks you right away. Very gassy. It's a cross between Pinesoul and Jet Fuel Gelato. I can honestly say it takes right after Pinesol because it smells just like the lemony cleaner.\n
Keyphrases: “Aroma, smell, lemon”\n
input text: First off the bag appeal ain't there but that's OK.\n
Keyphrases: “Bag appeal, packaging”\n

\n<</SYS>>\n\n"""

context = ""
user_prompt = ''' Please generate keyphrases only for those themes that are relevant to the content of the input text in the form of a python list where
each elements are separated by commas without any explanations.\n[/INST]'''
results = []

start_time = time.time() # Calculating start time
for feedback in flowers_df['Content with Quotes Cleansed Translated Clauses'][:1000]:
  context = "input text: "+feedback+"\n[/INST]"
  if feedback != 'Empty':
    sequences = pipeline(
        sys_prompt+context,
        do_sample=True,
        top_k=40,
        temperature=0.5,
        top_p=0.1,
        repetition_penalty=1.176,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=300,
    )
    for seq in sequences:
        gen_response = seq['generated_text']
        latest_index = gen_response.rfind('[/INST]')
        result = gen_response[latest_index + len('[/INST]'):]
        print("Text:", feedback)
        print(f"Result: {result}")
        results.append(result)
        # print(gen_response)
        print("------------------End------------------")
  else:
    results.append('Empty')

end_time = time.time()
print("Execution Time:", end_time - start_time)

Text: I saw other people post nice pictures of this product  and bought a 3 5g from Pointe Claire SQDC of 48 North Green Crush  August 2021
Result:   Sure! Here are some keyphrases that could accurately summarize the main points of the input text:

* "Product review"
* "Nice pictures"
* "Buying experience"
* "Pointe Claire SQDC"
* "48 North Green Crush"
* "August 2021"
------------------End------------------
Text: It was predominantly leaves and stems with no crystals
Result:   Here are some keyphrases that could accurately summarize the main points of the input text:

* Leaves
* Stems
* No crystals
------------------End------------------
Text: I even tested it
Result:   Here are some potential keyphrases that could accurately summarize the main ideas present in the input text:

* "Testing"
* "Tried it out"
* "Experience with product"
* "Product review"
------------------End------------------
Text: it had less than 6  advertised as 19  29   THC
Result:   Here are some potential keyphra

In [ ]:
import pickle
with open('drive/MyDrive/Novacene/fewshot_keyphrases_v1.pkl', 'wb') as f:
   pickle.dump(results, f)

##### Parse output to list of keywords

In [ ]:
len(results)

1000

In [ ]:
sys_prompt = """<s>[INST]<<SYS>>\n
You will be given an AI generated response which consists of bullet points.
Your task is to extract the information in the bullet points alone and give it in the form of a list of elements separated by commas. Please don't give any explanations and only output the list of elements.

\n<</SYS>>\n\n"""

context = ""
user_prompt = ''' Please output thhe list of elements directly without any extra filler content in your response".\n[/INST]'''
keyword_results = []

start_time = time.time() # Calculating start time
for feedback in results:
  context = feedback+"\n[/INST]"
  if feedback != 'Empty':
    sequences = pipeline(
        sys_prompt+context+user_prompt,
        do_sample=True,
        top_k=40,
        temperature=0.5,
        top_p=0.1,
        repetition_penalty=1.176,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=300,
    )
    for seq in sequences:
        gen_response = seq['generated_text']
        latest_index = gen_response.rfind('[/INST]')
        result = gen_response[latest_index + len('[/INST]'):]
        print("Text:", feedback)
        if len(result.split('\n')) == 3:
          print("Result:", result.split('\n')[2])
          keyword_results.append(result.split('\n')[2])
        elif len(result.split('\n')) == 2:
          print("Result:", result.split('\n')[1])
          keyword_results.append(result.split('\n')[1])
        else:
          print("Result:", result.split('\n')[0])
          keyword_results.append(result.split('\n')[0])
        # print(gen_response)
        print("------------------End------------------")
  else:
    keyword_results.append('Empty')

end_time = time.time()
print("Execution Time:", end_time - start_time)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Text:   Sure! Here are some keyphrases that could accurately summarize the main points of the input text:

* "Product review"
* "Nice pictures"
* "Buying experience"
* "Pointe Claire SQDC"
* "48 North Green Crush"
* "August 2021"
Result:   Product review, Nice pictures, Buying experience, Pointe Claire SQDC, 48 North Green Crush, August 2021
------------------End------------------
Text:   Here are some keyphrases that could accurately summarize the main points of the input text:

* Leaves
* Stems
* No crystals
Result: Leaves, Stems, Crystals
------------------End------------------
Text:   Here are some potential keyphrases that could accurately summarize the main ideas present in the input text:

* "Testing"
* "Tried it out"
* "Experience with product"
* "Product review"
Result: Testing, Tried it out, Experience with product, Product review
------------------End------------------
Text:   Here are some potential keyphrases that could accurately summarize the main points of the input tex

In [ ]:
keyword_results

In [ ]:
c = 0
for id, row in flowers_df.iterrows():
  if c == 1000:
    print(id)
    break
  if row['Content with Quotes Cleansed Translated Clauses'] != 'Empty':
    flowers_df['Content with Quotes Cleansed Translated Clauses'].loc[id] = row['Content with Quotes Cleansed Translated Clauses'] + "\n Keywords:"+ keyword_results[c]
    # print(row)
  c+=1

<ipython-input-18-b49b1125c932>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flowers_df['Content with Quotes Cleansed Translated Clauses'].loc[id] = row['Content with Quotes Cleansed Translated Clauses'] + "\n Keywords: "+ keyword_results[c]


1000


In [ ]:
flowers_df.to_csv("drive/MyDrive/Novacene/enriched_flowers_keywords_few_shot.csv")

In [ ]:
file = open("drive/MyDrive/Novacene/fewshot_keyphrases_v1.pkl",'rb')
results = pickle.load(file)

In [ ]:
len(results)

1296

In [ ]:
flowers_df['Content with Quotes Cleansed Translated Clauses'][2]

'It was predominantly leaves and stems with no crystals\n Keywords: Leaves, Stems, Crystals'